In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')

### Funciones

In [65]:
def fecha(data, dia):
    data["Date"] = data["ds"].str[0:8]
    data["Date"] = pd.to_datetime(data["Date"], format="%Y%m%d").dt.date
    data["Time"] = data["ds"].str[9:15] + "." + data["ds"].str[16:19]
    data["Time"] = pd.to_datetime(data["Time"], format="%H%M%S.%f").dt.time
    data["DateTime"] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].astype(str))
    data = data[data["DateTime"].dt.day == dia]
    data = data[["DateTime", "Price", "Volume"]]
    return data

def bid_ask(data):
    data["Price_shift"] = data["Price"].shift(-1)
    data["Price_diff"] = data["Price_shift"] - data["Price"]
    data["Price_diff_shift"] = data["Price_diff"].shift(1).fillna(0)
    data.loc[data["Price_diff_shift"]>0, "Bid_Ask_price_change"] = "Ask"
    data.loc[data["Price_diff_shift"]<0, "Bid_Ask_price_change"] = "Bid"
    return data

def vol_agg(data):
    c=0
    orden = []
    for x in data["Bid_Ask_price_change"].values:
        if (x=="Ask") | (x=="Bid"):
            c = c+1
            orden.append(str(c))
        else:
            orden.append(np.nan)
    data["orden"] = orden
    data["Bid_Ask"] = data["Bid_Ask_price_change"].fillna(method="ffill")
    data["Bid_Ask"].fillna("NS", inplace=True)
    data["orden"] = data["orden"].fillna(method="ffill")
    data["Bid_Ask_orden"] = data["Bid_Ask"]+"_"+data["orden"]
    data["Volume_agg"] = data.groupby("Bid_Ask_orden")["Volume"].transform(sum)
    data.loc[data["Volume_agg"].isnull(),"Volume_agg"] = data["Volume"]
    data.loc[data["Bid_Ask_price_change"].notnull(), "Volume_Final"] = data["Volume_agg"]
    print(data["Bid_Ask"].value_counts(dropna=False))
    return data

def seleccion(data, price_diff_min, vol_min):
    #data = data[(data["Volume_Final"].notnull()) | (data["Price_diff_shift"]!=0)]
    data = data.loc[(data["Volume_Final"]>vol_min) | (np.abs(data["Price_diff_shift"])>=price_diff_min)]
    print(data.shape)
    return data

def slope_dif(data):
    print(data.shape)
    #data["idx"] = range(len(data)) 
    data["max_min"] = data["maximum"].fillna(0) + data["minimum"].fillna(0)
    data["max_min"].replace(0,np.nan, inplace=True)
    
    dfmm = data[(data["max_min"].notnull()) & (data["Volume_Final"]!=0)]
    dfmm = data[(data["max_min"].notnull())]
    dfmm["maxmin_shift"] = dfmm["max_min"].shift(-1)
    dfmm["maxmin_diff"] = dfmm["maxmin_shift"] - dfmm["max_min"]
    dfmm["maxmin_diff_shift"] = dfmm["maxmin_diff"].shift(1).fillna(0)
    
    dfmm["seconds_diff"] = df["DateTime"].diff(1).dt.seconds
    dfmm["seconds_diff"].replace(0,0.01, inplace=True)
    #dfmm["idx_diff"] = dfmm["idx"].diff()
    dfmm["slope"] = 100*(dfmm["maxmin_diff_shift"]/dfmm["seconds_diff"])
    data=pd.merge(data, dfmm[['maxmin_diff','maxmin_diff_shift',"seconds_diff",'slope']], left_index=True, right_index=True, how="left")
    return data

def slope_points(data, slope, dif, field):
    mask_climax = (np.abs(data["maxmin_diff_shift"])>dif) & (np.abs(data["slope"])>slope)
    data.loc[mask_climax, "climax"] = data[field]
    ## puntos de climax
    index_list = list(data[data["climax"].notnull()].index)
    index_list_after = np.array(index_list)+int(window/2)
    index_list_after = index_list_after[index_list_after<len(data)]
    index_list_before = np.array(index_list)-int(window/2)
    index_list_before = index_list_before[index_list_before>0]
    data["after_climax"] = data.iloc[index_list_after][field]
    data["before_climax"] = data.iloc[index_list_before][field]
    return data
    

def vol_hist(data, freq):
    data = data.set_index('DateTime')
    data["Volume_freq"] = data["Volume_Final"].resample(freq).transform(sum)
    return data
   

def maximos(data, window, field):
    data.reset_index(inplace=True)
    data["maximum"] = data[field].rolling(window=window, center=True).max().rolling(window=window, center=True).max()
    data.loc[data["maximum"]!=data[field], "maximum"] = np.nan
    ## puntos de máximos
    index_list = list(data[data["maximum"].notnull()].index)
    index_list_after = np.array(index_list)+int(window/2)
    index_list_after = index_list_after[index_list_after<len(data)]
    index_list_before = np.array(index_list)-int(window/2)
    index_list_before = index_list_before[index_list_before>0]
    data["after_max"] = data.iloc[index_list_after][field]
    data["before_max"] = data.iloc[index_list_before][field]
    
    return data

def minimos(data, window, field):
    data.reset_index(inplace=True)
    data["minimum"] = data[field].rolling(window=window, center=True).min().rolling(window=window, center=True).min()
    data.loc[data["minimum"]!=data[field], "minimum"] = np.nan
    ## puntos de minimos
    index_list = list(data[data["minimum"].notnull()].index)
    index_list_after = np.array(index_list)+int(window/2)
    index_list_after = index_list_after[index_list_after<len(data)]
    index_list_before = np.array(index_list)-int(window/2)
    index_list_before = index_list_before[index_list_before>0]
    data["after_min"] = data.iloc[index_list_after][field]
    data["before_min"] = data.iloc[index_list_before][field]
    return data

def media(data, window, field):
    data["media_movil"] = data[field].rolling(window=window).mean()
    return data

#### PLOTTING #####

def plot_maximos(data, field):
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.plot(data["maximum"], "o", c="green")
    ax.plot(data[field], c="black")
    plt.show()
def plot_minimos(data, field):
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.plot(data["minimum"], "o", c="red")
    ax.plot(data[field], c="black")
    plt.show()
def plot_ask_bid(data, field, V):
    fig = plt.figure(figsize=(15,7))
    ax = fig.add_subplot(111)
    ax.plot(data[(data["Bid_Ask"]=="Ask") & (data["Volume_Final"]>=V)][field], "o", c="red", lw=10)
    ax.plot(data[(data["Bid_Ask"]=="Bid") & (data["Volume_Final"]>=V)][field], "o", c="green", lw=10)
    ax.plot(data[field], c="black", lw=0.5)
    plt.show()
def plot_ab_mm(data, field, V):
    fig = plt.figure(figsize=(15,7))
    ax = fig.add_subplot(111)
    ax.plot(data["minimum"], "^", c="orange")
    ax.plot(data["maximum"], "v", c="purple")
    ax.plot(data[(data["Bid_Ask"]=="Ask") & (data["Volume_Final"]>=V)][field], "o", c="red", markersize=10)
    ax.plot(data[(data["Bid_Ask"]=="Bid") & (data["Volume_Final"]>=V)][field], "o", c="green", markersize=10)
    ax.plot(data[field], c="black", lw=0.5)
    plt.show()
def plot_slope_mm(data, field, slope, dif):
    mask_climax = (np.abs(data["maxmin_diff_shift"])>dif) & (np.abs(data["slope"])>slope)
    fig = plt.figure(figsize=(15,7))
    ax = fig.add_subplot(111)
    ax.plot(data["minimum"], "^", c="orange", markersize=10)
    ax.plot(data["maximum"], "v", c="purple", markersize=10)
    ax.plot(data["after_climax"], "o", c="black")
    ax.plot(data["before_climax"], "o", c="black")
    ax.plot(data[mask_climax][field], ">", c="blue", markersize=10)
    ax.plot(data[field], c="black", lw=0.5)
    plt.show()
def plot_ab_mm_v(data, field, V, time_ini, time_fin, slope, dif, annotations):
    data = data[time_ini:time_fin] 
    data_bid = data[(data["Bid_Ask"]=="Bid") & (data["Volume_Final"]>=V)] 
    data_ask = data[(data["Bid_Ask"]=="Ask") & (data["Volume_Final"]>=V)] 
    
    fig, (ax0, ax1) = plt.subplots(ncols=1,nrows=2, figsize=(20,10), sharex=True,gridspec_kw={'height_ratios': [3,1]})
    ax0.plot(data["minimum"], "^", c="orange")
    ax0.plot(data["maximum"], "v", c="purple")
    ax0.plot(data["after_climax"], "o", c="black")
    ax0.plot(data["before_climax"], "o", c="black")
    ax0.scatter(x = data_ask.index, y = data_ask[field], c="green", s=10*data_ask["Volume_Final"], alpha=0.5)
    ax0.scatter(x = data_bid.index, y = data_bid[field], c="red", s=10*data_bid["Volume_Final"], alpha=0.5)
    if annotations == True:
        for i, txt in enumerate(data_ask["Volume_Final"].astype(int).values):
            ax0.annotate(txt, (data_ask.index.values[i-1], data_ask[field].values[i-1]))
        for i, txt in enumerate(data_bid["Volume_Final"].astype(int).values):
            ax0.annotate(txt, (data_bid.index.values[i-1], data_bid[field].values[i-1]))
    ax0.plot(data["climax"], ">", c="blue", markersize=10)
    ax0.plot(data["media_movil"], c="grey", lw=1)
    ax0.plot(data[field], c="black", lw=0.75)
    ax1.plot(data["Volume_freq"])
    plt.show()

## Lectura del fichero

In [66]:
#CL 19_12_22.Last
df_last = pd.read_csv("CL 19_12_22.Last.txt", sep=";", names=["ds", "low", "high", "Price", "Volume"])
print(df_last.shape)
df_last.head(2)

(155599, 5)


,ds,low,high,Price,Volume
0,20221218 230000 0000000,74.62,74.62,74.62,1
1,20221218 230000 0000000,74.62,74.62,74.62,5


### Ejecución de los cálculos

In [67]:
window=20 ## 20 y 20 a cada lado de los máximos y mínimos (fractales) -> naranja y magenta
price_diff_min = 0.015 # cambio de precio mínimo con el que nos quedamos
vol_min = 5 # volumen agregado mínimo con el que nos quedamos
freq = "20s" #1min # frecuencia del histograma de volumen
dif = 0.3 # cambio de precio entre fractales (triángulo azul)
slope = 20.0 # pendiente entre fractales --> ejemplo: slope = 1, mueve 20 puntos en 20 segundos (triángulo azul)

df1 = df_last.copy()
df = fecha(df1, 19)
df = df1.copy()
df = bid_ask(df)
df = vol_agg(df)
df = seleccion(df, price_diff_min, vol_min)
df = maximos(df, window, "Price")
df = minimos(df, window, "Price")
df = slope_dif(df)
df = slope_points(df, slope, dif, "Price")
df = media(df, 200, "Price")
df = vol_hist(df, freq)

Ask    83360
Bid    72231
NS         8
Name: Bid_Ask, dtype: int64
(12144, 17)
(12144, 25)


In [68]:
%matplotlib qt 
V = 20 # volumen mínimo para pintar los ask y los bid (verdes y rojo)
time_ini = "2022-12-19 08:40:00"
time_fin = "2022-12-19 13:30:00"
plot_ab_mm_v(df, "Price", V, time_ini, time_fin, slope, dif, False)

In [69]:
df.reset_index().to_csv("slope_analysis.csv", sep=";", decimal=",", index=False)

In [70]:
time_ini = "2022-12-19 11:25:00"
time_fin = "2022-12-19 11:30:00"
df[time_ini:time_fin][["Volume_Final","Bid_Ask"]]

,Volume_Final,Bid_Ask
DateTime,,
2022-12-19 11:25:11.527,7.0,Bid
2022-12-19 11:25:41.093,6.0,Ask
2022-12-19 11:25:44.902,7.0,Bid
2022-12-19 11:25:52.284,6.0,Bid
2022-12-19 11:26:44.478,4.0,Bid
2022-12-19 11:26:51.261,4.0,Bid
2022-12-19 11:27:20.718,1.0,Ask
2022-12-19 11:27:39.635,6.0,Bid
2022-12-19 11:28:00.847,12.0,Ask
